# ワードクラウド
ここでは、テンプレートを使ってワードクラウドを作成し、頻度差がある単語の可視化を行います。


In [1]:
## 必要なライブラリの読み込み
import sys
import os
import re
import math
import glob
import pandas as pd

テキストの前処理で作成した処理後（単語分割後）のファイルを使い、どちらのカテゴリ(H1,H2)に、どの単語が、何回出現したかをカウントします。Pandasを利用することで、簡単に集計できます。

In [9]:
### 教師データ：カテゴリ H1 H2 ごとの重複ありでの全部の単語が入る
### 列名はデータにはなく（header=None）、"W"という名前の列名を付ける。
raw_words = {}
raw_words['H1'] = pd.read_table('mlit.txt',encoding='utf-8',header=None,names=('W'))
raw_words['H2'] = pd.read_table('env.txt',encoding='utf-8',header=None,names=('W'))

### カテゴリ H1 H2 ごと、各単語の出現回数
word_count = {}
word_count['H1'] = raw_words['H1']['W'].value_counts()
word_count['H2'] = raw_words['H2']['W'].value_counts()   ### word_count['H1'].get('事業',0) 等で回数を使える

### カテゴリ H1 H2 両方を通しての全部の出現単語（重複無し）
all_words = pd.concat([raw_words['H1'],raw_words['H2']])['W'].unique()

print(type(raw_words['H1']))
print(raw_words['H1'])
print(raw_words['H1']['W'])
print(raw_words['H1']['W'][1])
print(len(raw_words['H1']['W']))
print(type(word_count['H1']))
print(word_count['H1'])
print(word_count['H1']['事業'])
print(type(all_words))
print(all_words)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


<class 'pandas.core.frame.DataFrame'>
           W
0        我が国
1         人口
2         減少
3         社会
4        迎える
5         高齢
6         速い
7       スピード
8         進む
9         産業
10       支える
11        生産
12        年齢
13        人口
14        減少
15       続ける
16        財政
17        制約
18        国際
19        競争
20        環境
21        厳し
22        増す
23        経済
24       支える
25        労働
26        減少
27        する
28        社会
29        ムダ
...      ...
155684    評価
155685    実施
155686    する
155687    維持
155688    管理
155689    分野
155690    現場
155691    検証
155692    一定
155693    性能
155694    確認
155695    する
155696    技術
155697    点検
155698    同等
155699    環境
155700    試行
155701    する
155702    実用
155703    検証
155704    する
155705    図表
155706    II
155707   次世代
155708    社会
155709  インフラ
155710  ロボット
155711    開発
155712    導入
155713    推進

[155714 rows x 1 columns]
0          我が国
1           人口
2           減少
3           社会
4          迎える
5           高齢
6           速い
7         スピード
8        

pandas の動作を確認します。下記print文を下の枠に入れて実行してみて下さい。<br>
read_table するとDataFrameオブジェクト、value_counts()の戻り値はSeriesオブジェクト、<br>
DataFrameのunique()を呼び出すとndarray、と、タイプが色々違ってきますので、使い方に注意が必要です。<br>
- `print(type(raw_words['H1']))`
- `print(raw_words['H1'])`
- `print(raw_words['H1']['W'])`
- `print(raw_words['H1']['W'][1])`
- `print(len(raw_words['H1']['W']))`
- `print(type(word_count['H1']))`
- `print(word_count['H1'])`
- `print(word_count['H1']['事業'])`
- `print(type(all_words))`
- `print(all_words)`

In [10]:
print(type(raw_words['H1']))

<class 'pandas.core.frame.DataFrame'>


可視化の為、ある単語がどちらのカテゴリにどの程度出現しているか、をリスト化します。
（これは単語ごとの尤度に相当します）<br>
カテゴリH1に単語W1が出現する確率(L1)  = H1での単語W1の出現数 / H1の総単語数<br>
同様にカテゴリH2に単語W1が出現する確率(L2)を算出します。<br>
上記で作った all_of_words に出現する単語全てが揃っているので、その単語全部に対しリスト化
偏りを見たいので、L1 - L2 をキーにして降順、昇順、両方向で並べ替えをします。
L1 - L2 = 0 の場合、H1,H2で出現比率が同じなので、偏りがない。L1 - L2 > 0 なら、
H1での出現率が高く、マイナスならH2での出現率の方が高い。


In [11]:
word_stat = []
for w in all_words:
    word_stat.append([ w, \
        word_count['H1'].get(w,0)/len(raw_words['H1']) - word_count['H2'].get(w,0)/len(raw_words['H2']), \
        word_count['H1'].get(w,0), \
        word_count['H2'].get(w,0) ] )

threshold_count = 10   # 最低10回以上出現した単語を対象にします

# word_stat の 2番目の要素(x[1]で指定)を使って、降順(word_stat_1)、昇順(word_stat_2)に並べ替えたものを用意。
# 合わせて最低出現回数(threshold_count)を設け、それ以下の数回しか登場しないものを除いておく。
word_stat_1 = sorted([z for z in word_stat if z[2]>=threshold_count and z[1]>0 ], key = lambda x: - x[1] )
word_stat_2 = sorted([z for z in word_stat if z[3]>=threshold_count and -z[1]>0 ], key = lambda x: x[1] )

out_str_1 = ''
for i in range(min(200,len(word_stat_1))):
    out_str_1 += "  {'text': '" + word_stat_1[i][0] + "', 'size': " + str(word_stat_1[i][1]) + "},\n"

out_str_2 = ''
for i in range(min(200,len(word_stat_2))):
    out_str_2 += "  {'text': '" + word_stat_2[i][0] + "', 'size': " + str(-word_stat_2[i][1]) + "},\n"

print(out_str_1[:300] + "...\n")   # 作った文字列の最初の方だけ表示してみる

  {'text': '図表', 'size': 0.006768819759302311},
  {'text': '交通', 'size': 0.006308499701470625},
  {'text': '整備', 'size': 0.005747852370554493},
  {'text': '取組む', 'size': 0.005280670315013894},
  {'text': '国土', 'size': 0.0049401222415133604},
  {'text': 'インフラ', 'size': 0.0042321592200351525},
  {'tex...



上で作成したデータを、ワードクラウドのテンプレートに差し込んで、ワードクラウドを作ります。

In [12]:
with open('wordcloud_template.html', 'r', encoding='utf-8') as f_template:
    template = f_template.read()

with open('wc1.html', 'w', encoding='utf-8') as wordcloud1:
    wordcloud1.write( template.replace("$$DATA$$",out_str_1,1) )

with open('wc2.html', 'w', encoding='utf-8') as wordcloud2:
    wordcloud2.write( template.replace("$$DATA$$",out_str_2,1) )

glob.glob('wc?.html')


['wc2.html', 'wc1.html']